In [ ]:
import pandas as pd
import numpy as np
import os
import math

"""
Iris classification based on provided data
Author: Paweł Kozikowski
"""

NEURAL_NETWORK_DATABASE: str = 'weights.npz'

def sigmoid(value):
      return ((1 / (1 + (np.exp(-value)))))
def sigmoid_derivative(value):
      return ((np.exp(-value) / ((1 + np.exp(-value)) ** 2)))

def read_csv_data(csv_file: str) -> list[dict]:
      df = pd.read_csv(os.path.abspath(csv_file))
      df_format_to_dict: list[dict] = df.to_dict(orient="records")
      
      return df_format_to_dict

def format_csv_data(csv_data: dict) -> list[tuple[np.ndarray, np.ndarray]]:
      formatted_data_array: list[tuple[np.ndarray, np.ndarray]] = []
      
      for data_pack in csv_data:
            input_matrix: np.ndarray = np.array([
                  np.float64(data_pack["SepalLengthCm"]),
                  np.float64(data_pack["SepalWidthCm"]),
                  np.float64(data_pack["PetalLengthCm"]),
                  np.float64(data_pack["PetalWidthCm"])
            ])
            
            label_matrix: np.ndarray = np.array([])

            if (data_pack["Species"] == "Iris-setosa"):
                  label_matrix = np.array([1, 0, 0])
            elif (data_pack["Species"] == "Iris-versicolor"):
                  label_matrix = np.array([0, 1, 0])
            elif (data_pack["Species"] == "Iris-virginica"):
                  label_matrix = np.array([0, 0, 1])

            tupled_data: tuple[np.ndarray, np.ndarray] = (input_matrix, label_matrix)
            formatted_data_array.append(tupled_data)
      
      return formatted_data_array

def save_neural_network_parameters(npz_file: str, parameters: tuple[np.ndarray, np.ndarray]) -> None:
      new_weights: np.ndarray = parameters[0]
      new_biases: np.ndarray = parameters[1]

      np.savez(npz_file, 
               weights=new_weights,
               biases=new_biases)
      
      return None

def load_neural_network_parameters(npz_file: str) -> tuple[np.ndarray, np.ndarray]:
      loaded_neural_network_data = np.load(npz_file)
      loaded_weights: np.ndarray = loaded_neural_network_data["weights"]
      loaded_biases: np.ndarray = loaded_neural_network_data["biases"]

      return loaded_weights, loaded_biases

def front_propagation(input_matrix: np.ndarray, weights_matrix: np.ndarray, biases_matrix: np.ndarray):
      linear_output = input_matrix @ weights_matrix + biases_matrix
      return linear_output

def back_propagation(input_matrix: np.ndarray, sums_matrix: np.ndarray, target_matrix: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
      sigmoid_matrix: np.ndarray = sigmoid(sums_matrix)
      sigmoid_derivative_matrix: np.ndarray = sigmoid_derivative(sums_matrix)
      
      OT_matrix: np.ndarray = sigmoid_matrix - target_matrix
      OT_matrix = -2 * OT_matrix

      gradient_weights_matrix: np.ndarray = input_matrix.T @ sigmoid_derivative_matrix
      gradient_weights_matrix: np.ndarray = gradient_weights_matrix * OT_matrix
      gradient_bias_matrix: np.ndarray = sigmoid_derivative_matrix * OT_matrix

      return gradient_weights_matrix, gradient_bias_matrix
      
def single_train(input_matrix: np.ndarray, target_matrix: np.ndarray, learning_reat: np.float64 = 0.1) -> None:
      """
      single_train FUNCTION

      Param:
            input_matrix: Matrix containg input values for our neural network
            target_matrix: Matrix containg prepared target values informing which answer is correct
            learning_rate: Rate with which our neural network learns (Can be interpreted as jump in looking for minimum in our cost function)

      Returns:
            None

      Purpose:
            Adjust weights and biases of our Neural Network to perform better at presenting us correct answers
      """ 
      # Load current weights and biases of our neural network
      weights_matrix, biases_matrix = load_neural_network_parameters(NEURAL_NETWORK_DATABASE)
      # Front propagation
      sums_matrix: np.ndarray = front_propagation(input_matrix, weights_matrix, biases_matrix)
      
      # Back propagation
      weights_gradient, biases_gradient = back_propagation(input_matrix, sums_matrix, target_matrix)
      # Calculating new weights and biases for our neural network
      new_weights_matrix: np.ndarray = weights_matrix - learning_reat * weights_gradient
      new_biases_matrix: np.ndarray = biases_matrix - learning_reat * biases_gradient

      # Saving new data in database
      save_neural_network_parameters(NEURAL_NETWORK_DATABASE, (new_weights_matrix, new_biases_matrix))

random_weights = np.random.rand(4, 3)
print("Random weights shape:", random_weights.shape)
print(random_weights)
random_inputs = np.random.rand(4)
print("Random inputs shape:", random_inputs.shape)
print(random_inputs)
random_biases = np.random.rand(3)
print("Random biases shape:", random_biases.shape)
print(random_biases)

print("Front propagation results: ")
print(front_propagation(random_inputs, random_weights, random_biases))

Random weights shape: (4, 3)
[[0.19345059 0.60766925 0.20824328]
 [0.31627351 0.79991851 0.54929838]
 [0.26063442 0.13703967 0.34359184]
 [0.22606951 0.9341868  0.21422426]]
Random inputs shape: (4,)
Random inputs shape transponse (4,)
[0.33027158 0.05187004 0.43370551 0.46907143]
Random biases shape: (3,)
[0.56986068 0.40977493 0.70589063]
Front propagation results: 
[0.86923837 1.14959782 1.05266375]
